# **Исследование надежности заемщиков**

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку
<br><br>
Основная цель проекта: определить как различные данные клиента банка такие как - количество детей, семейное положение, уровень дохода и разные цели кредита влияют на его возврат в срок. Эта информация понадобится в дальнейшем для определения основных критерий, по каторому будут выдаваться кредиты
<br><br>
Для начала исходную таблицу нужно привести в такой вид, чтобы с ней было комфортно работать. Для этого необходимо произвести предобоработку данных, чтобы затем перейти к анализу данных с целью обнаружения закономерностей, полностью отвечали на вопросы, поднятые в основной цели проекта

## **Шаг 1. Обзор данных**

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
display (df.head(10))
df.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<b>Комментарии:</b>
<br>  
Открыли таблицу, дали ей имя `df`, изучили общую информацию. Таблица имеет 21525 строк и 12 столбцов. 
<br><br> 
При беглом анализе данных в таблице df обнаружилось:
<br>
- В столбцах `days_employed` и `total_income` есть пропуски;
<br>
- В `days_employed` есть отрицательные значения;
<br>
- В `education` есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв;
<br>
- В `purpose` находятся неявные дубликаты
<br> <br>     
После данного анализа делается вывод, что перед началом исследования необходима предобработка данных во избежании искажений последующих результатов
<br>    
</div>



## Шаг 2.1. Заполнение пропусков

### *Поиск пропусков по строкам в исходной таблице*

**Вывод количества пропусков по столбцам при помощи команды *isna()* :**

In [2]:
display(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В столбцах `days_employed` и `total_income` обнаружили одинаковое количество пропусков. Нужно проверить есть ли в этом закономерность
<br><br>

In [3]:
miss_value = df[(df['days_employed'].isna()) & (df['total_income'].isna())] 

print('Количество пустых строк в столбце "days_employed" и "total_income" составляет:', miss_value.shape[0])

Количество пустых строк в столбце "days_employed" и "total_income" составляет: 2174


Закономерность обнаружена: где есть пропуски в `days_employed`, то там есть пропуски и в `total_income` и наоборот. Проверим какая доля пропущенных значений в этих строках от общего значения, чтоб понимать, что дальше с ними делать         
<br>

**Определение доли пропущенных значений в столбцах `days_employed` и в `total_income`:**

In [4]:
conversion = len(df[df['days_employed'].isna()]) / (df.shape[0])
print()

print('Доля пропущенных значений в столбце "days_employed" и "total_income" с пропусками составляет: {:.5f}'.format(conversion))
print()

print('Доля отрицательных значений от общего числа значений в столбце "days_employed" составляет:', 
len(df[df['days_employed']<=0]) / df.shape[0])
print()

print('Доля отрицательных значений от общего числа значений в столбце "total_income" составляет:', 
len(df[df['total_income']<=0]) / df.shape[0])
print()


Доля пропущенных значений в столбце "days_employed" и "total_income" с пропусками составляет: 0.10100

Доля отрицательных значений от общего числа значений в столбце "days_employed" составляет: 0.7389547038327526

Доля отрицательных значений от общего числа значений в столбце "total_income" составляет: 0.0



Доля пропущенных значений в столбцах `days_employed` и `total_income` существенна, поэтому нужно определиться какими значениями нужно заполнять пропуски. Просто удалить строки нельзя иначе есть высокий шанс исказить  последующие результаты. 

Начать заполнять пропуски можно лишь после того, как определимся откуда появились аномалии (отрицательные значения) в столбце `days_employed`. В столбце `total_income` отрицательных значений нет
<br><br>

<div class="alert">
<b>Вывод по подразделу:</b>
    
В столбцах `days_employed` и `total_income` в одинаковых по номеру строках есть пропущенные значения, а их доля от общего числа значений весьма велика (около 0,101 или ~10%). Исходя из этого, а также отталкиваясь от своего собственного опыта можно сделать вывод о том, что в исходной таблице ежемесячный доход зависит от общего трудового стажа
<br><br>
Судя по тому, что столбец `days_employed` имеет формат float64 (см. Шаг 1), то можно сделать вывод о том, что с целью автоматизации расчетов к этому столбцу была применена формула вида:
<br><br>
<b>`days_employed  = (дата на момент оформления заявки на кредит) - (дата официального трудоустройства)`, </b><br> 
<br>
где вычитатель и вычитаемое в этой формуле представлены с точностью как минимум до часа, исходя из того, что у значений в столбце `days_employed` есть дробная часть
<br>  
В столбце `days_employed`, есть отрицательные значения. Это возможно либо потому что сотрудник, либо клиент банка перепутали во время подачи заявки на кредит графы "дата на момент оформления заявки на кредит" и "дата официального трудоустройства", либо попросту произошел сбой программы при подсчете. Окончательный вывод о том, откуда возникла данная аномалия стоит сделать лишь после того, как узнаем истинную причину у того, кто составлял эту таблицу. Но так как сейчас сделать это невозможно, поэтому просто уберем знак "-" со всех значений в рассматриваемом столбце, считая, что это произошла ошибка при подсчете. Также стоит учесть, что доля пропущенных значений в этих столбцах существенная (0,101), то строки, в которых есть пустые значения, ни в коем случае нельзя удалять во избежание существенных искажений последующих результатов.
<br>      
Заполнить пропуски в `days_employed` пока рано, т.к. в этом столбце, как говорилось ранее, есть отрицательные значения. Избавимся от них, а затем уже заполним пропуски в столбцах `days_employed` и `total_income`. Заполнять пропуски будем медианным значением, так как оно более устойчиво чем, например, среднее значение или мода к аномалиям из-за сильного разброса входящих данных (возраст клиентов, стаж, уровень заработка и т.д.).
<br>     
</div>

### Шаг 2.2 Проверка данных на аномалии и исправления.


Избавимся от отрциательных значений в столбце `days_employed`, взяв все значения по модулю при помощи команды **abs()** :

In [5]:
df['days_employed'] = abs(df['days_employed'])

Аномалия с отрицательным трудовым стажем устранена. Теперь проверим насколько адекватны максимальные значения стажа:

In [6]:
display(df.sort_values(by='days_employed', ascending = False).head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба


400000 дней стажа - это 1095 лет. В этом столбце безусловно есть ошибка. Проверим какая доля данных аномалий  есть в рассматриваемом столбце от общего количества значений, чтобы далее сделать вывод о том, что с этими значениями далбше делать. Аномальным количеством лет стажа будем считать число, которое больше предельно возможного (если работать с 18 лет до 70). 

In [7]:
print('Доля аномальных значений в столбце "days_employed":', (df[df['days_employed']>=(70-18)*365].shape[0]) / df.shape[0])

Доля аномальных значений в столбце "days_employed": 0.16004645760743322


Около 16% значений аномальны. Заменим их на максимальный возможный стаж, затем пропуски в столбце `days_employed` заменим на медианное значение

In [8]:
df.loc[df['days_employed'] >= (70-18)*365, 'days_employed'] = (70-18)*365
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median()) 
print('Медианное значение для столбца "days_employed" равно:', df['days_employed'].median())

Медианное значение для столбца "days_employed" равно: 2194.220566878695


Далее поэтапно будем искать уникальные значения в осавшихся столбцах:

In [9]:
print('Уникальные значения в столбце "children":', df['children'].unique())

Уникальные значения в столбце "children": [ 1  0  3  2 -1  4 20  5]


Аномальные значения в столбце "children" - это "-1" и "20". Найдем сколько этих значений и заменим их на "1" и "2" соответственно в случае, если значений "-1" и "20" не много относительно общего количества строк в рассматриваемом столбце

In [10]:
print ('Количество аномальных значений в столбце "children":',df.loc[df['children'] == -1].shape[0] + df.loc[df['children'] 
== 20].shape[0]) 

Количество аномальных значений в столбце "children": 123


Заменяем значения  "-1"  и  "20"  на  "1"  и  "2"  соответственно:

In [11]:
df.loc[df['children'] == -1, 'children'] = 1
df.loc[df['children'] == 20, 'children'] = 2

Идем дальше:

In [12]:
print('Уникальные значения в столбце "dob_years":', df['dob_years'].sort_values().unique())

Уникальные значения в столбце "dob_years": [ 0 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41
 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65
 66 67 68 69 70 71 72 73 74 75]


Аномальное значения в столбце `dob_years` (возраст клиента в годах) - это "0". Посмотрим сколько их:

In [13]:
print ('Количество аномальных значений в столбце "dob_years":', df.loc[df['dob_years'] == 0].shape[0])

Количество аномальных значений в столбце "dob_years": 101


Количество аномальных значений в столбце `dob_years` не критично, поэтому заменим эту аномалию на среднее значение:

In [14]:
df.loc[df['dob_years'] == 0, 'dob_years'] = df['dob_years'].mean()

Идем дальше:

In [15]:
print('Уникальные значения в столбце "education":', df['education'].unique())

Уникальные значения в столбце "education": ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']


В этом столбце далее нужно будет все значения свести к одному регистру и сформировать к нему дополнительный датафрейм словарь

In [16]:
print('Уникальные значения в столбце "family_status":', df['family_status'].unique())

Уникальные значения в столбце "family_status": ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']


В этом столбце есть примерно такая же проблема как и в столбце "education" - разный регистр, поэтому здесь также далее сведем все к одному регистру, а далее - сформируем к нему дополнительный датафрейм словарь

In [17]:
print('Уникальные значения в столбце "family_status_id":', df['family_status_id'].unique())

Уникальные значения в столбце "family_status_id": [0 1 2 3 4]


In [18]:
print('Уникальные значения в столбце "gender":', df['gender'].unique())

Уникальные значения в столбце "gender": ['F' 'M' 'XNA']


Странный гендер "XNA", выбивающийся из ряда других значений в рассматриваемом столбце. Нужно вывести на экран количество строк со значением "XNA" в столбце "gender", чтоб узнать на сколько критична эта аномалия:

In [19]:
print('Количество уникальных значений в столбце "gender":', df[df['gender']=='XNA'].shape[0])

Количество уникальных значений в столбце "gender": 1


Одно уникальзное значение. Оставим его как есть, т.к. оно не сильно исказит последующие расчеты и результаты

In [20]:
print('Уникальные значения в столбце "income_type":', df['income_type'].unique())
print('Уникальные значения в столбце "debt":', df['debt'].unique())

Уникальные значения в столбце "income_type": ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
Уникальные значения в столбце "debt": [0 1]


Здесь все нормально, без аномалий
<br><br>
В столбце `total_income` заполним медианным значением пустые строки, как обсуждалось раннее:

In [21]:
df['total_income'] = df['total_income'].fillna(df['total_income'].median())
print('Медианное значение для столбца "total_income" равно:', df['days_employed'].median())

Медианное значение для столбца "total_income" равно: 2194.220566878695


In [22]:
print('Уникальные значения в столбце "purpose":', df['purpose'].unique())

Уникальные значения в столбце "purpose": ['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим обр

В столбце `purpose` нужно будет избавиться от неявных дубликатов, как предполагалось ранее
<br>
<br>
Ниже подведем итоги анализа

<div class="alert">
<b>Выводы по подразделу:</b>
<br><br>
Подведем итоги после поиска аномалий в таблице "df" :
    
- в столбце "days_employed" отрицательные показатели были заменены на теже значения взятые по модулю;
<br>   
- в столбце "days_employed" неестественно большие показатели были заменены на макисмально возможные;
<br> 
- в столбце "days_employed" все пропуски были заменены на медианное значение;
<br>  
- в столбце "children" заменили значения "-1" и "20" на значения "1" и "2" соответственно. Этих аномальных значений немного, поэтому в случае ошибочного рассуждения данная замена не сильно исказит последующие исследования; 
<br>
- в столбце "education" и "family_status" есть значения записанные с использованием заглавных и строчных букв. Далее нужно будет привести их всех к одному регистру;
<br>
- в столбце "dob_years" значение "0" заменили на медиану; 
<br>
- в столбце "gender" обнаружена выбивающееся значение "XNA". Оно попалось один раз, пока ничего с ним не делаем;
<br>
- в столбце "total_income" пропуски заменили медианным значением;
<br>
- в столбце "purpose" нужно будет далее избавиться от неявных дубликатов путем создания функции, которая на основании предложенных данных сформирует новый столбец "purpose_category", в который войдут следующие категории: "операции с автомобилем", "операции с недвижимостью", "проведение свадьбы", "получение образования";
<br>
- все преобразования в выше указанных столбцах сильно не повлияют на конечный результат исследования, поэтому все эти изменения более чем уместны;
<br><br>      
</div>

### Шаг 2.3. Изменение типов данных.

In [23]:
# В столбцах 'total_income' и 'days_employed' числовые значения имеют дробную часть. Уберем ее при помощи команды astype('int') 
# с целью улучшения визуализации всей таблицы

df['total_income'] = df['total_income'].astype('int')
df['days_employed'] = df['days_employed'].astype('int')

### Шаг 2.4. Удаление дубликатов.

**Поиск неявных дубликатов**

In [24]:
# В столбцах 'education' и family_status' разный регистр. Сведем все значения из этих столбцов к одному низкому регистру 
# при помощи команды str.lower():

df['education'] = df['education'].str.lower()
print (df['education'].unique())
df['family_status'] = df['family_status'].str.lower()
print (df['family_status'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']


**Поиск явных дубликатов**

In [25]:
print(df.duplicated().sum()) 

71


Обнаружено 71 дубликат в исходной таблице. Выведем таблицу с этими дубликатами:

In [26]:
duplicated_df = df[df.duplicated()].head(10) #результат — датафрейм с дубликатами
display(duplicated_df) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,2194,41.0,среднее,1,женат / замужем,0,F,сотрудник,0,145017,покупка жилья для семьи
3290,0,2194,58.0,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу
4182,1,2194,34.0,высшее,0,гражданский брак,1,F,сотрудник,0,145017,свадьба
4851,0,2194,60.0,среднее,1,гражданский брак,1,F,пенсионер,0,145017,свадьба
5557,0,2194,58.0,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу
6312,0,2194,30.0,среднее,1,женат / замужем,0,M,сотрудник,0,145017,строительство жилой недвижимости
7808,0,2194,57.0,среднее,1,гражданский брак,1,F,пенсионер,0,145017,на проведение свадьбы
7921,0,2194,64.0,высшее,0,гражданский брак,1,F,пенсионер,0,145017,на проведение свадьбы
7938,0,2194,71.0,среднее,1,гражданский брак,1,F,пенсионер,0,145017,на проведение свадьбы
8583,0,2194,58.0,высшее,0,не женат / не замужем,4,F,пенсионер,0,145017,дополнительное образование


Явные дубликаты обнаружены в столбцах `days_employed` и `total_income`, где значения соответственно равны **2194** и **145017**. Стоит отметить, что остальные значения в таблице `duplicated_df` столбцах различны, а это лишь еще раз доказывает, что ошибка заключается лишь в столбцах `days_employed` и `total_income` по неясной на то причине
<br><br>
Из-за небольшого количества обнаружнных строк с явными дубликатами относительно размера всей изначальной таблицы, а также в связи с отсутствием возможности узнать истинную причину возникшей аномалии у составителя данного датафрейма, единственным верным решением будет - удалить строки с повторяющимися значениями

In [27]:
df = df.drop_duplicates().reset_index(drop=True) # создадим обновленную df таблицу, очищенную от дубликатов, 
# с новой индексацией и удалением старой. Командой ниже проверим результат удаления строк с дубликатами

display(df[df.duplicated()].head(10)) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Явных дубликатов не обнаружено

<div class="alert">
<b>Выводы по подразделу:</b>
<br><br>
    
- В столбцах `education` и `family_status` свели все значения к одному низкому регистру при помощи команды str.lower():
<br>
- В столбцах `days_employed` и `total_income` обнаружили явные дубликаты в количестве 71 штук. Из-за невозможности узнать природу возникновения данных дубликатов, а также в связи их относительно небольшого количества приняли единственное возможное решение - удалить строки с этими дубликатами, обновив после этого индиксацию таблицы `df`
<br><br>      
</div>


### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [28]:
#Создадим два новых датафрейма-словаря, в которых:
# каждому уникальному значению из education соответствует уникальное значение education_id — в первом;
# каждому уникальному значению из family_status соответствует уникальное значение family_status_id — во втором;

# Для кажого нового датафрейма сначала соединим два выбранных столбца из исходной таблицы, затем там удалим дубликаты и сортируем
# по возрастанию значений в столбце с id
# Новые датафреймы выведем на экран


education_log = df[['education', 'education_id']]
education_log = education_log.drop_duplicates().reset_index(drop=True)
family_status_log = education_log.sort_values(by='education_id', ascending = False)
display(education_log.head())
print()


family_status_log = df[['family_status', 'family_status_id']]
family_status_log = family_status_log.drop_duplicates().reset_index(drop=True)
family_status_log = family_status_log.sort_values('family_status_id', ascending = False)
display(family_status_log.head())
print()

# Удалим столбцы 'education', 'family_status' из-за их ненадобности в дальнейшем

df.drop(columns = ['education', 'family_status'], axis = 1,  inplace=True) 
df.columns

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


,family_status,family_status_id
4,не женат / не замужем,4
3,в разводе,3
2,вдовец / вдова,2
1,гражданский брак,1
0,женат / замужем,0


Index(['children', 'days_employed', 'dob_years', 'education_id',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose'],
      dtype='object')

### Шаг 2.6. Категоризация дохода.

In [29]:
# При помощи функции определим категоризацию дохода в зависимости от ЗП клиента:
def total_income_category (income):
    
    if income <= 30000:
        return 'E'
    if 30001 <= income <= 50000:
        return 'D'
    if 50001 <= income <= 200000:
        return 'C'
    if 200001 <= income <= 1000000:
        return 'B'
    if income >= 1000001:
        return 'A'

# Для проверки выведем несколько пробных значений:
    
    
print(total_income_category(10000))
print(total_income_category(45000))  
print(total_income_category(100000))  
print(total_income_category(1000000)) 
print(total_income_category(10000000))  

# Добавим новый столбец 'total_income_category' в таблицу "df", значения будут привязаны к значениям столбца 'total_income'

df['total_income_category'] = df['total_income'].apply(total_income_category)
display(df.head())

E
D
C
B
A


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42.0,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33.0,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,18980,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

In [30]:
# Как говорилось раннее, в столбце 'purpose_category' есть неявные дубликаты. Устраним эту аномалию, при помощи функции, которая 
# для категоризации целей кредита будет искать общие слова в значениях 'purpose_category'

def purpose_category (purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'недвиж' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью' 
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образован' in purpose:
        return 'получение образования'
    return 'категория не определена'

# Добавим новый столбец 'purpose_category' в таблицу "df", значения будут привязаны к значениям столбца 'purpose'

df['purpose_category'] = df['purpose'].apply(purpose_category)
display(df.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42.0,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33.0,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,18980,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


### Ответы на вопросы.

##### Вопрос 1: 

**Есть ли зависимость между количеством детей и возвратом кредита в срок?**

In [31]:
# Для того, чтобы ответить на вопрос "Есть ли зависимость между количеством детей и возвратом кредита в срок", 
# нужно воспользоваться методом pivot_table(). где:

#index — столбец или столбцы, по которым группируют данные (количество детей у клиента)
#columns — столбец, по значениям которого происходит группировка (имел ли задолженность по возврату кредитов)
#values — значения, по которым мы хотим увидеть сводную таблицу (например, семейный статус)
#aggfunc — функция, применяемая к значениям (количество клиентов в зависимости от кол-ва у них детей)

data_pivot = df.pivot_table(index=['children'], columns='debt', values='family_status_id', aggfunc='count')
display(data_pivot) 



debt,0,1
children,,
0,13028.0,1063.0
1,4410.0,445.0
2,1926.0,202.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


Первоначально может показаться, судя по выгруженной таблице, что чем больше детей у клиента банка тем больше количество задолжностей и вовремя выплаченных кредитов. Но это не совсем так, ведь людей, у которых нет детей или их 1-2, куда больше чем тех, у кого их более чем 3.
<br><br>
Поделим значения из первого и второго столбца на общее количество клиентов, чтобы в дальнейшем было возможно сравнивать полученные показатели и сделать вывод о том, есть ли зависимость между количеством детей и возвратом кредита в срок




In [32]:
data_pivot['share_debt_0'] = data_pivot[0] / df.shape[0]
data_pivot['share_debt_1'] = data_pivot[1] / df.shape[0]
display(data_pivot) 

# Найдем самых благонадежных для банка в зависимости от количества у них детей:

data_pivot['possitive'] = (data_pivot[0]) /  (data_pivot[0] + data_pivot[1])
display(data_pivot) 


debt,0,1,share_debt_0,share_debt_1
children,,,,
0,13028.0,1063.0,0.607253,0.049548
1,4410.0,445.0,0.205556,0.020742
2,1926.0,202.0,0.089773,0.009415
3,303.0,27.0,0.014123,0.001259
4,37.0,4.0,0.001725,0.000186
5,9.0,NaN,0.000420,NaN


debt,0,1,share_debt_0,share_debt_1,possitive
children,,,,,
0,13028.0,1063.0,0.607253,0.049548,0.924562
1,4410.0,445.0,0.205556,0.020742,0.908342
2,1926.0,202.0,0.089773,0.009415,0.905075
3,303.0,27.0,0.014123,0.001259,0.918182
4,37.0,4.0,0.001725,0.000186,0.902439
5,9.0,NaN,0.000420,NaN,NaN


**Вывод:** 
<br>
- Клиент банка, который чаще всего берет кредит - клиент без детей;
- Самые благонадежные клиенты банка - это люди, у которых нет детей;
- Самые не благонадежные клиенты банка - это люди, у которых один и более ребенок



##### Вопрос 2: 

**Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [33]:
data_pivot1 = df.pivot_table(index=['family_status_id'], columns='debt', values='children', aggfunc='count')
display(data_pivot1) 

# Проделаем такой же план действий как и в первом вопросе:

data_pivot1['share_debt_0'] = data_pivot1[0] / df.shape[0]
data_pivot1['share_debt_1'] = data_pivot1[1] / df.shape[0]
display(data_pivot1) 

data_pivot1['possitive'] = (data_pivot1[0]) /  (data_pivot1[0] + data_pivot1[1])
display(data_pivot1) 


debt,0,1
family_status_id,,
0,11408,931
1,3763,388
2,896,63
3,1110,85
4,2536,274


debt,0,1,share_debt_0,share_debt_1
family_status_id,,,,
0,11408,931,0.531742,0.043395
1,3763,388,0.175399,0.018085
2,896,63,0.041764,0.002937
3,1110,85,0.051739,0.003962
4,2536,274,0.118206,0.012772


debt,0,1,share_debt_0,share_debt_1,possitive
family_status_id,,,,,
0,11408,931,0.531742,0.043395,0.924548
1,3763,388,0.175399,0.018085,0.906529
2,896,63,0.041764,0.002937,0.934307
3,1110,85,0.051739,0.003962,0.928870
4,2536,274,0.118206,0.012772,0.902491


**Вывод:** 
<br>
- Чаще всего берут кредит - женатые люди;
- Самые благонадежные клиенты - вдовцы, затем в разводе, т.е. это люди, которые когда-то уже были женаты, но на момент взятия кредита холосты. Видимо это те люди, которые отчетливо понимают зачем им нужен кредит и сколько денег для этого нужно взять в связи с их богатым жизненным опытом;
- Самые не благонадежные клиенты - не женатые люди, затем те, кто в гражданском браке. Скорее всего это люди молодого возраста, судя по тому, что их доля от количества всех клиентов банка не велика, и видимо они только начинают свою кредитную историю



##### Вопрос 3: 

**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [34]:
data_pivot2 = df.pivot_table(index=['total_income_category'], columns='debt', values='children', aggfunc='count')
display(data_pivot2) 

# Проделаем такой же план действий как и в первом вопросе:

data_pivot2['share_debt_0'] = data_pivot2[0] / df.shape[0]
data_pivot2['share_debt_1'] = data_pivot2[1] / df.shape[0]
display(data_pivot2) 

data_pivot2['possitive'] = (data_pivot2[0]) /  (data_pivot2[0] + data_pivot2[1])
display(data_pivot2) 

debt,0,1
total_income_category,,
A,23,2
B,4685,356
C,14656,1360
D,329,21
E,20,2


debt,0,1,share_debt_0,share_debt_1
total_income_category,,,,
A,23,2,0.001072,0.000093
B,4685,356,0.218374,0.016594
C,14656,1360,0.683136,0.063391
D,329,21,0.015335,0.000979
E,20,2,0.000932,0.000093


debt,0,1,share_debt_0,share_debt_1,possitive
total_income_category,,,,,
A,23,2,0.001072,0.000093,0.920000
B,4685,356,0.218374,0.016594,0.929379
C,14656,1360,0.683136,0.063391,0.915085
D,329,21,0.015335,0.000979,0.940000
E,20,2,0.000932,0.000093,0.909091


**Вывод:** 
<br>
- Кредит в подавляющем количестве относительно других клиентов берут люди с уровнем зарплат "С"
- Клиенты с низким уровнем зарплат ("Е" и "D"), а также люди с высоким уровнем достатка("А") почти не обращаются в банк, поэтому по ним трудно сделать какие-то однозначные выводы;
- Исходя из всего вышеперечисленного, делается вывод о том, что самыми благонадежными клиентами однозначно являются клиенты с уровнем зарплат выше среднего ("С" и "В")



##### Вопрос 4: 

**Как разные цели кредита влияют на его возврат в срок?**

In [35]:
data_pivot3 = df.pivot_table(index=['purpose_category'], columns='debt', values='children', aggfunc='count')
display(data_pivot3) 

# Проделаем такой же план действий как и в первом вопросе:

data_pivot3['share_debt_0'] = data_pivot3[0] / df.shape[0]
data_pivot3['share_debt_1'] = data_pivot3[1] / df.shape[0]
display(data_pivot3) 

data_pivot3['possitive'] = (data_pivot3[0]) /  (data_pivot3[0] + data_pivot3[1])
display(data_pivot3) 

debt,0,1
purpose_category,,
операции с автомобилем,3903,403
операции с недвижимостью,10029,782
получение образования,3643,370
проведение свадьбы,2138,186


debt,0,1,share_debt_0,share_debt_1
purpose_category,,,,
операции с автомобилем,3903,403,0.181924,0.018784
операции с недвижимостью,10029,782,0.467465,0.036450
получение образования,3643,370,0.169805,0.017246
проведение свадьбы,2138,186,0.099655,0.008670


debt,0,1,share_debt_0,share_debt_1,possitive
purpose_category,,,,,
операции с автомобилем,3903,403,0.181924,0.018784,0.906410
операции с недвижимостью,10029,782,0.467465,0.036450,0.927666
получение образования,3643,370,0.169805,0.017246,0.907800
проведение свадьбы,2138,186,0.099655,0.008670,0.919966


**Вывод:** 
<br>
- Чаще всего кредит берут клиенты для операции с недвижимостью;
- Самые благонадежные клиенты - клиенты, которые берут кредит для операции с недвижимостью
- Самые не благонадежные клиенты - клиенты, которые берут кредит для операции с автомобилем или получения образования

## Общий вывод:

- Самый благонадежный клиент банка - клиент, который ранее был женат, не имеет детей, выше среднего достатка, а кредит он берет с целью проведения операции с недвижимостью;

- Самый не благонадежный клиент банка - клиент, который не женат или в гражданском браке, есть дети, ниже среднего достатка, который берет кредит с целью проведения операции с автомобилем или получения образования
<br><br>

В результате данного анализа делается вывод о том, что семейное положение и количество детей напрямую зависит на факт погашения кредита в срок